In [100]:
import pandas as pd
import pymysql as mc 
import os
import sys
import time
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import getpass
from functools import reduce

In [188]:
pw = getpass.getpass()
cnx = mc.connect(user='akashgoyal',password=pw,
                 host='stromberg.cs.uchicago.edu',db='mlb_practicum',port=3306)

········


In [3]:
query = "SELECT * FROM prior_win_probability"
df_priors = pd.read_sql_query(query,cnx)

In [189]:
year = ['2011','2012']
query = """SELECT Game.gameID AS gameID,homeTeamScore,awayTeamScore,curr_inn,balls,strikes,
                        firstBaseRunner, secondBaseRunner,thirdBaseRunner,home_team_runs,away_team_runs, outs
                        FROM Pitch2 INNER JOIN Game ON Game.gameID = Pitch2.gameID 
                        WHERE YEAR(gameDate) IN ({})""".format(','.join(year))
df = pd.read_sql_query(query,cnx)

In [104]:
def win_or_lose(df):
    if df['curr_inn']*2%2==1:
        if df['homeTeamScore']>df['awayTeamScore']:
            return 1
        return 0
    else:
        if df['awayTeamScore']>df['homeTeamScore']:
            return 1
        return 0
    
def current_lead(df):
    lead = df['home_team_runs']-df['away_team_runs']
    if df['curr_inn']*2%2 == 1:
        return lead
    return -lead

In [107]:
#if you are calculating posterior probabilities, these need to be independent
df['runner1B'] = df.apply(lambda row: 1 if row['firstBaseRunner'] else 0,axis=1)
df['runner2B'] = df.apply(lambda row: 1 if row['secondBaseRunner'] else 0,axis=1)
df['runner3B'] = df.apply(lambda row: 1 if row['thirdBaseRunner'] else 0,axis=1)

In [190]:
df['win'] = df.apply(win_or_lose,axis=1)
df['curr_lead'] = df.apply(current_lead,axis=1)
df['current_inning'] = df.apply(lambda row: int(row['curr_inn']),axis=1)

In [95]:
#if running regression do this.
def runner_1b_scen(df):
    if df['firstBaseRunner']:
        if not df['secondBaseRunner']:
            if not df['thirdBaseRunner']:
                return 1
    return 0

def runner_2b_scen(df):
    if not df['secondBaseRunner']:
        return 0 
    if not df['firstBaseRunner']:
        if not df['thirdBaseRunner']:
            return 1
    return 0

def runner_3b_scen(df):
    if df['thirdBaseRunner']:
        if not df['secondBaseRunner']:
            if not df['firstBaseRunner']:
                return 1
    return 0

def runner_1b_2b_scen(df):
    if df['firstBaseRunner']:
        if df['secondBaseRunner']: 
            if not df['thirdBaseRunner']:
                return 1
    return 0

def runner_1b_3b_scen(df):
    if df['firstBaseRunner']:
        if df['thirdBaseRunner']: 
            if not df['secondBaseRunner']:
                return 1
    return 0
def runner_2b_3b_scen(df):
    if df['secondBaseRunner']:
        if df['thirdBaseRunner']:
            if not df['firstBaseRunner']:
                return 1
    return 0
def runner_loaded_scen(df):
    if df['firstBaseRunner']:
        if df['secondBaseRunner']:
            if df['thirdBaseRunner']:
                return 1
    return 0

def runner_empty(df):
    if not df['firstBaseRunner']:
        if not df['secondBaseRunner']:
            if not df['thirdBaseRunner']:
                return 1
    return 0

In [191]:
funcs = [runner_1b_scen,runner_2b_scen,runner_3b_scen,
         runner_1b_2b_scen,runner_1b_3b_scen,runner_2b_3b_scen,runner_loaded_scen,runner_empty]
names = ['1B','2B','3B','12B','13B','23B','123B','empty_bases']
for i,function in enumerate(funcs):
    df[names[i]] = df.apply(function,axis=1)

In [108]:
df2 = df.copy()
df2 = df2.drop(columns=['homeTeamScore','awayTeamScore','curr_inn','balls','strikes','firstBaseRunner',
                        'secondBaseRunner','thirdBaseRunner','home_team_runs','away_team_runs'])

In [109]:
df2.head()

,gameID,outs,win,curr_lead,current_inning,runner1B,runner2B,runner3B
0,ANA201104080,0,1,0,1,0,0,0
1,ANA201104080,0,1,0,1,0,0,0
2,ANA201104080,0,1,0,1,0,0,0
3,ANA201104080,0,1,0,1,0,0,0
4,ANA201104080,1,1,0,1,0,0,0


In [110]:
for col in df2.columns:
    if col=='gameID':
        continue
    print (col)

outs
win
curr_lead
current_inning
runner1B
runner2B
runner3B


In [182]:
#example should have the prior probability
def calculate_posterior_prob(df,example):
    num_probs = []
    denom_probs = []
    rows = df.shape[0]
    
    cond_df = df[df['win']==1]
    total_wins = cond_df.shape[0]
    
    p_y = total_wins/df.shape[0]
    
    for col in df.columns:
        if col == 'gameID':
            #print ('got a gameID')
            continue
        elif col == 'win':
            #print ('at win')
            continue
        #p(X|win)
        p_x_given_y_num = cond_df[cond_df[col] == example[col]].shape[0] #p(runner on first)|won
        total_denom = total_wins
        numerator = 0
        if p_x_given_y_num == 0:
            numerator = 0
        else:
            numerator = p_x_given_y_num/total_denom
        num_probs.append(numerator)
        #print ('num',numerator)
        
        #p(X)
        p_x = df[df[col]==example[col]].shape[0]
        denominator = p_x/rows 
        denom_probs.append(denominator)
        #print ('denom',denominator)
    
    numerator_total = reduce(lambda x,y: x*y, num_probs)
    denominator_total = reduce(lambda x,y: x*y, denom_probs)
    return numerator_total*p_y/denominator_total
    

In [184]:
for i in range(129,160):
    example1 = df2.iloc[i]
    print(calculate_posterior_prob(df2,example1))

0.2232702951767628
0.2424530637977973
0.2424530637977973
0.2424530637977973
0.787794153103597
0.787794153103597
0.787794153103597
0.787794153103597
0.787794153103597
0.787794153103597
0.787794153103597
0.9140385962572631
0.9140385962572631
0.9140385962572631
0.9140385962572631
0.9140385962572631
0.9925702741448962
0.9925702741448962
0.9925702741448962
0.9925702741448962
0.9925702741448962
0.9925702741448962
0.9925702741448962
1.0101185301930447
1.0101185301930447
1.0101185301930447
1.0101185301930447
0.9975903239162449
0.9975903239162449
0.9975903239162449
0.1895062551752655


In [186]:
df2.iloc[150:160]

,gameID,outs,win,curr_lead,current_inning,runner1B,runner2B,runner3B
150,ANA201104080,0,0,2,5,1,1,0
151,ANA201104080,0,0,2,5,1,1,0
152,ANA201104080,1,0,2,5,1,1,0
153,ANA201104080,1,0,2,5,1,1,0
154,ANA201104080,1,0,2,5,1,1,0
155,ANA201104080,1,0,2,5,1,1,0
156,ANA201104080,2,0,2,5,1,1,0
157,ANA201104080,2,0,2,5,1,1,0
158,ANA201104080,2,0,2,5,1,1,0
159,ANA201104080,0,1,-2,6,0,0,0
